In [1]:
%pylab inline
import graphlab as gl
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:

train_dir = './training_images/'
try:
    os.mkdir(train_dir)
except:
    pass

In [3]:
training_set_sf = gl.SFrame.read_csv('train.csv')

[INFO] This non-commercial license of GraphLab Create is assigned to jackelb@mcmaster.caand will expire on October 31, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-5916 - Server binary: C:\Users\Ben\Anaconda\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\Ben\AppData\Local\Temp\graphlab_server_1446595557.log.0
[INFO] GraphLab Server Version: 1.6.1


PROGRESS: Finished parsing file C:\Kagle Competitions\Right Whale Recognition\train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.019014 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file C:\Kagle Competitions\Right Whale Recognition\train.csv
PROGRESS: Parsing completed. Parsed 4544 lines in 0.01401 secs.


In [4]:
intmap = dict([(i, val) for i, val in zip(training_set_sf['Image'], training_set_sf['whaleID'])])

In [5]:
import shutil

In [16]:
for i in training_set_sf['Image']:
    shutil.copyfile("./imgs/" + i, train_dir+i)

In [6]:
def encode_data(y):
    intmap = dict([(val, i) for i, val in enumerate(set(y))]) 
    return array([intmap[val] for val in y])

In [24]:
try:
    train = gl.SFrame('./sframe/train_shuffle')
except:
    train_sf = gl.image_analysis.load_images(train_dir, random_order=True)
    train_sf['image'] = gl.image_analysis.resize(train_sf['image'], 256, 256)
    #Extract the image name from the path
    train_sf['w_id'] = train_sf['path'].apply(lambda x: intmap[x.split('/')[-1]])
    train_sf.save('./sframe/train_shuffle')

PROGRESS: Read 1880 images in 5.01259 secs	speed: 375.056 file/sec
PROGRESS: Read 3790 images in 10.0131 secs	speed: 381.957 file/sec


In [7]:
gl.canvas.set_target('ipynb')

In [8]:

train.head()

path,image,w_id
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_37654
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_27820
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_35004
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_70138
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_82602
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_99243
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_79439
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_65586
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_47734
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,whale_48386


In [7]:
unique_labels = train['w_id'].unique().sort()                                              
class_map = {}                                                                  
for i in range(len(unique_labels)):                                                        
    class_map[unique_labels[i]] = i                                                        
train['label'] = train['w_id'].apply(lambda x: class_map[x])

# Save the mapping so that we can use it later when building customized classifier
import pickle
pickle.dump(class_map, file('./w_id_to_label.pkl', 'w'))

NameError: name 'train' is not defined

In [8]:
mean_image = train['image'].mean()
gl.SArray([mean_image]).save('./sframe/mean_image')
mean_image = gl.SArray('./sframe/mean_image')[0]

NameError: name 'train' is not defined

net = gl.deeplearning.get_builtin_neuralnet('imagenet')
net

net.layers[-2].num_hidden_units=447
net.verify(input_shape=[256,256,3], output_shape=447)

m = gl.neuralnet_classifier.create(train[['image', 'label']],                      
                                   target='label',
                                   network=net,                                    
                                   mean_image=mean_image,                          
                                   metric=['accuracy','recall@5'],                 
                                   max_iterations=35,                              
                                   model_checkpoint_path='./result/model_checkpoint',
                                   model_checkpoint_interval=5,                    
                                   batch_size=150)

m.save('./DeepNN_solution')

In [9]:
pretrained_model = gl.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')

PROGRESS: Downloading http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45/dir_archive.ini to C:/Users/Ben/AppData/Local/Temp/graphlab-Ben/5916/616834bb-a103-450c-82f9-bd5212ff4929.ini
PROGRESS: Downloading http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45/objects.bin to C:/Users/Ben/AppData/Local/Temp/graphlab-Ben/5916/e50c268e-4206-486f-a792-33ced831efa8.bin


In [12]:
try:
    X = gl.SFrame('./sframe/extracted_training_set')
except:
    X = train
    X['features'] = pretrained_model.extract_features(X)
    X.save('./sframe/extracted_training_set')

X_train, X_test = X.random_split(0.8)

X_train.head()

In [ ]:
#simple_classifier = graphlab.classifier.create(X, features = ['features'], target = 'w_id')

In [20]:
simple_lc = gl.logistic_classifier.create(X, features=['features'], target='w_id', lbfgs_memory_level=15, max_iterations=15)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: WARNING: Detected extremely low variance for feature(s) 'features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 4293
PROGRESS: Number of classes           : 445
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 4096
PROGRESS: Number of coefficients    : 1819068
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration

In [21]:
simple_lc.save('./logistic_classifier_model')

In [22]:
#model = gl.neuralnet_classifier.create(X_train, target='w_id', features = ['image'], validation_set=X_test )


In [76]:
sample_sub = gl.SFrame.read_csv("sample_submission.csv")

PROGRESS: Finished parsing file C:\Kagle Competitions\Right Whale Recognition\sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.088062 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,

In [26]:
try:
    p1_sf = gl.SFrame('./sframe/unlabeled_data1')
except:
    p1_sf = gl.image_analysis.load_images('./img_to_predict1')
    p1_sf['image'] = gl.image_analysis.resize(p1_sf['image'], 256, 256)
    p1_sf['features'] = pretrained_model.extract_features(p1_sf)
    p1_sf.save('./sframe/unlabeled_data1')

In [27]:
try:
    p1_sf = gl.SFrame('./sframe/unlabeled_data2')
except:
    p2_sf = gl.image_analysis.load_images('./img_to_predict2')
    p2_sf['image'] = gl.image_analysis.resize(p2_sf['image'], 256, 256)
    p2_sf['features'] = pretrained_model.extract_features(p2_sf)
    p2_sf.save('./sframe/unlabeled_data2')

In [28]:
try:
    p3_sf = gl.SFrame('./sframe/unlabeled_data3')
except:
    p3_sf = gl.image_analysis.load_images('./img_to_predict3')
    p3_sf['image'] = gl.image_analysis.resize(p3_sf['image'], 256, 256)
    p3_sf['features'] = pretrained_model.extract_features(p3_sf)
    p3_sf.save('./sframe/unlabeled_data3')

In [29]:
try:
    p4_sf = gl.SFrame('./sframe/unlabeled_data4')
except:
    p4_sf = gl.image_analysis.load_images('./img_to_predict4')
    p4_sf['image'] = gl.image_analysis.resize(p4_sf['image'], 256, 256)
    p4_sf['features'] = pretrained_model.extract_features(p4_sf)
    p4_sf.save('./sframe/unlabeled_data4')

In [30]:
predict_sf = p1_sf.append(p2_sf).append(p3_sf).append(p4_sf)

In [31]:
predict_sf.head()

path,image,features
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0,0.449148476124, 0.0, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0, 0.0, 0.0, 0.0,1.19997322559, 0.0, 0.0, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 1.6715233326, 0.0, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0, 0.0, 0.0, 0.0,1.25266504288, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0, 0.0, 0.0, 0.0,4.19807338715, 0.0, 0.0, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0, 0.0, 0.0,0.047479569912, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.14406812191, 0.0,0.610118687153, 0.0, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0, 0.0, 0.0,1.270829916, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0, 0.0, 0.0,0.955793082714, ..."
C:/KagleCompetitions/Right Wh ...,Height: 256 Width: 256,"[0.0, 0.0, 0.0, 0.0, 0.0,1.39044833183, 0.0, 0.0, ..."


In [33]:
simple_lc = gl.load_model('logistic_classifier_model/')
predictions = simple_lc.predict(predict_sf)

In [34]:
predictions.head()

dtype: str
Rows: 10
['whale_99243', 'whale_70138', 'whale_11242', 'whale_08923', 'whale_38302', 'whale_34663', 'whale_41805', 'whale_52749', 'whale_28892', 'whale_82089']

In [36]:
unique_labels = set(training_set_sf['whaleID'])
vector_map = dict([i[::-1] for i in enumerate(unique_labels)])

In [37]:
labels = list(unique_labels)

In [38]:
test_sub_list = []
for i in range(len(predict_sf['path'])):
    out_vector = list(zeros(len(unique_labels), dtype=int))
    out_vector[vector_map[predictions[0]]] = 1
    #out_vector = [X['path'][i].split('/')[-1]] + out_vector
    test_sub_list.append(out_vector)

In [39]:
test_sub_list = array(test_sub_list)
test_sub = gl.SFrame({'Image':[i.split('/')[-1] for i in predict_sf['path']]})
for i in range(len(labels)):
    test_sub.add_column(gl.SArray(test_sub_list[:,i]), name=labels[i])

In [40]:
test_sub.save('test_sub.csv', format='csv')

In [14]:
import glob
total_files = glob.glob('./imgs/*.jpg')
total_files = [i.split('\\')[-1] for i in total_files]
training_files = training_set_sf['Image']
files_to_predict = [x for x in total_files if x not in training_files]

In [20]:
n_files = len(files_to_predict)

In [24]:
fp1 = files_to_predict[:n_files/4]
fp2 = files_to_predict[n_files/4:n_files/2]
fp3 = files_to_predict[n_files/2:3*n_files/4] 
fp4 = files_to_predict[3*n_files/4:]

In [25]:

try:
    os.mkdir('./img_to_predict1')
    os.mkdir('./img_to_predict2')
    os.mkdir('./img_to_predict3')
    os.mkdir('./img_to_predict4')
except:
    pass
for i in fp1:
    shutil.copyfile("./imgs/" + i, './img_to_predict1/'+i)
for i in fp2:
    shutil.copyfile("./imgs/" + i, './img_to_predict2/'+i)
for i in fp3:
    shutil.copyfile("./imgs/" + i, './img_to_predict3/'+i)
for i in fp4:
    shutil.copyfile("./imgs/" + i, './img_to_predict4/'+i)
    